# Twitter Sentiment Analysis and Preprocessing Pipeline

Authenticate

In [ ]:
api_key = 'xxxxxxxxxxxxxxxxx'
api_key_secret = 'xxxxxxxxxxxxxxxxxx'
acess_token = 'xxxxxxxxxxxxxxxxxx'
acess_token_secret = 'xxxxxxxxxxxxxxxxxxxxx'

#Authenticate
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

Extracting code

In [ ]:
import tweepy
import csv
from datetime import datetime, timedelta

# Define keywords for scraping tweets
keywords = ['keyword']

# Define the time period (last 14 days)
since_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

# Define the number of tweets to fetch per keyword
num_tweets = 10

# List to store scraped tweets
tweets = []

# Perform the search query
for keyword in keywords:
    query = f"{keyword} since:{since_date} -filter:retweets"
    searched_tweets = tweepy.Cursor(
        api.search_tweets, q=query, lang="en", tweet_mode="extended"
    ).items(num_tweets)

    for tweet in searched_tweets:
        tweets.append(tweet)

# Save the tweets to a CSV file
with open("EOS.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["tweet", "created_at"])

    for tweet in tweets:
        writer.writerow([tweet.full_text.replace('\n', ' '), tweet.created_at])


NLP labelling

In [ ]:
import pandas as pd
from textblob import TextBlob

# Set the path for the input CSV file
input_file = "/content/.csv"  # Path to the CSV

# Load the input CSV file
df = pd.read_csv(input_file)

# Create a new column for sentiment labels
def get_sentiment_label(tweet):
    blob = TextBlob(tweet)
    polarity = blob.sentiment.polarity
    if polarity > 0.4:
        return 'Positive'
    elif polarity > 0:
        return 'Moderately Positive'
    elif polarity == 0:
        return 'Neutral'
    elif polarity > -0.4:
        return 'Moderately Negative'
    else:
        return 'Negative'

df['Sentiment'] = df['tweet'].apply(get_sentiment_label)

# Add an ID column
df['id'] = df.index + 1

# Reorder the columns
df = df[['id', 'created_at', 'tweet', 'Sentiment']]

# Display all columns side by side
pd.set_option('display.max_colwidth', None)

# Save the new CSV file
output_file = input_file.replace(".csv", "_labeled.csv")
df.to_csv(output_file, index=False)

count labels

In [ ]:
# load the labelled tweets CSV file into a DataFrame
df_tweets = pd.read_csv('/content/Rose_labeled.csv')

# count the number of positive, negative, and neutral tweets
counts = df_tweets['Sentiment'].value_counts()

# print the counts
print(counts)

Converting Shortwords

In [ ]:
import pandas as pd

# Read the CSV file
df_tweets = pd.read_csv('/content/EOS_labeled.csv')  # Input path

# Section: Short Words Mapping
# Define a dictionary to map short words to their full form
short_words = {
    "u": "you", "ur": "your", "r": "are", "ya": "you all", "121": "one to one",
    "a/s/l": "age, sex, location", "adn": "any day now", "afaik": "as far as I know",
    "afk": "away from keyboard", "aight": "alright", "alol": "actually laughing out loud",
    "b4": "before", "b4n": "bye for now", "bak": "back at the keyboard",
    "bf": "boyfriend", "bff": "best friends forever", "bfn": "bye for now",
    "bg": "big grin", "bta": "but then again", "btw": "by the way"
}

# Section: Replace Short Words
def replace_short_words(tweet):
    """Replace short words in a tweet with their full form."""
    if not isinstance(tweet, str):
        return tweet
    words = tweet.split()
    words = [short_words[word] if word in short_words else word for word in words]
    return " ".join(words)

# Apply the function to the "tweet" column
df_tweets["tweet"] = df_tweets["tweet"].apply(replace_short_words)

# Section: Save Cleaned Dataset
# df_tweets.to_csv('short_word_labelled_tweets.csv', index=False)  # Save as CSV


Replacing Contraction

In [ ]:
import re

# Define apostrophe dictionary for contractions
apostrophes = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "could've": "could have",
    "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not",
    "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he's": "he is",
    "i'm": "I am", "isn't": "is not", "it's": "it is", "let's": "let us", "she's": "she is",
    "that's": "that is", "there's": "there is", "they're": "they are", "we're": "we are",
    "weren't": "were not", "what's": "what is", "where's": "where is", "who's": "who is",
    "you're": "you are", "you've": "you have"
}

# Replace contractions with full words
def replace_apostrophes(tweet):
    if isinstance(tweet, str):
        for old, new in apostrophes.items():
            tweet = tweet.replace(old, new)
    return tweet

df_tweets['tweet'] = df_tweets['tweet'].apply(replace_apostrophes)


Clean Tweet Text

In [ ]:
# Remove RT, special characters, and convert to lowercase
def clean_tweet(tweet):
    tweet = re.sub(r'RT @\w+: ', '', tweet)  # Remove retweets
    tweet = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet)  # Remove mentions, links, and special characters
    return tweet.lower()

df_tweets['tweet'] = df_tweets['tweet'].apply(clean_tweet)

# Remove duplicate tweets
df_tweets.drop_duplicates(subset='tweet', inplace=True)

Spell Correction

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_misspelled_words(tweet):
    words = tweet.split()
    corrected_words = [spell.correction(word) if spell.unknown([word]) else word for word in words]
    return ' '.join(filter(None, corrected_words))

df_tweets['tweet'] = df_tweets['tweet'].apply(correct_misspelled_words)


Stopword Removal

In [ ]:
import nltk
nltk.download('stopwords')

stopwords = set(nltk.corpus.stopwords.words('english'))
def remove_stop_words(tweet):
    return ' '.join([word for word in tweet.split() if word not in stopwords])

df_tweets['tweet'] = df_tweets['tweet'].apply(remove_stop_words)

Tokenization and Lemmatization

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
df_tweets['tokens'] = df_tweets['tweet'].apply(word_tokenize)
df_tweets['lemmatized'] = df_tweets['tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])


In [ ]:
df_tweets.to_csv('EOS_tweets.csv', index=False)
